In [52]:
%run code/import_libs.py
%run code/functions.py
%run code/Build_Forms.py
%matplotlib inline


pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 30)
pd.options.display.float_format = '{:,}'.format


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [54]:
df=pd.read_csv('data/kc_house_data.csv', index_col=0)

# initial cleaning operations (splitting , fillig na, etc)

df["grade"]=df["grade"].map(lambda x: x.split()[0]).astype(int)
df['condition'] = df['condition'].apply(enum_category)
df['waterfront'].fillna(value='NO',inplace=True)

#FILTER
#df=df[df['yr_built']>1980]
df=df[df['price']<1500000].copy()
df=df[df['sqft_living']<7000].copy()
df=df[df['sqft_lot']<100000].copy()
df=df[~df["grade"].isin([3,4,5])]



In [55]:
initial_pred = df.drop(columns=["price"]).copy()
initial_price = df[["price"]]

x_train, x_test, y_train, y_test = train_test_split(initial_pred, initial_price, test_size=0.3)

x_train_fin, y_train_fin = transform_data(x_train, y_train)
x_test_fin, y_test_fin = transform_data (x_test, y_test)

linreg = LinearRegression()
linreg.fit(x_train_fin, y_train_fin)

y_train_pred = linreg.predict(x_train_fin)
y_test_pred = linreg.predict(x_test_fin)

test_mse = mean_squared_error(np.exp(y_test_fin), np.exp(y_test_pred))
print('Test Mean Squarred Error:', '{:,}'.format(np.sqrt(test_mse)))

train_mse = mean_squared_error(np.exp(y_train_fin), np.exp(y_train_pred))
print('Train Mean Squarred Error:', '{:,}'.format(np.sqrt(train_mse)))

Test Mean Squarred Error: 0.9162239925217436
Train Mean Squarred Error: 0.8706896137075374


In [45]:
# Create OLS linear model
pred=x_train_fin
price=y_train_fin
pred_int = sm.add_constant(pred)
model = sm.OLS(price,pred_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.882
Model:                            OLS   Adj. R-squared:                  0.880
Method:                 Least Squares   F-statistic:                     543.6
Date:                Mon, 28 Mar 2022   Prob (F-statistic):               0.00
Time:                        16:48:41   Log-Likelihood:                -5110.1
No. Observations:               14592   AIC:                         1.062e+04
Df Residuals:                   14393   BIC:                         1.213e+04
Df Model:                         198                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -8.8257      0.103    -85.387      0.000      -9.028      -8.623
sqft_living       0.8796      0.011     77.176      0.000       0.857       0.902
sqft_lot          0.1432      0.006     22.753      0.000       0.131       0.156
gra_7             0.2074      0.012     17.190      0.000       0.184       0.231
gra_8             0.4123      0.015     28.344      0.000       0.384       0.441
gra_9             0.6542      0.018     36.507      0.000       0.619       0.689
gra_10            0.8535      0.022     38.503      0.000       0.810       0.897
gra_11            1.0802      0.030     36.481      0.000       1.022       1.138
gra_12            1.3350      0.057     23.419      0.000       1.223       1.447
gra_13            1.9615      0.176     11.165      0.000       1.617       2.306
zip_98002         0.0240      0.038      0.639      0.523      -0.050       0.097
zip_98003         0.0875      0.033      2.626      0.009       0.022       0.153
zip_98004         2.2480      0.033     67.345      0.000       2.183       2.313
zip_98005         1.5008      0.040     37.375      0.000       1.422       1.580
zip_98006         1.3613      0.030     45.461      0.000       1.303       1.420
zip_98007         1.3745      0.042     32.429      0.000       1.291       1.458
zip_98008         1.4012      0.034     41.282      0.000       1.335       1.468
zip_98010         0.5045      0.052      9.679      0.000       0.402       0.607
zip_98011         0.9341      0.037     25.010      0.000       0.861       1.007
zip_98014         0.4914      0.050      9.822      0.000       0.393       0.589
zip_98019         0.6475      0.041     15.978      0.000       0.568       0.727
zip_98022         0.0760      0.038      1.982      0.047       0.001       0.151
zip_98023         0.0087      0.030      0.293      0.769      -0.049       0.067
zip_98024         0.8415      0.073     11.507      0.000       0.698       0.985
zip_98027         1.0819      0.032     34.040      0.000       1.020       1.144
zip_98028         0.8424      0.033     25.205      0.000       0.777       0.908
zip_98029         1.2278      0.033     37.683      0.000       1.164       1.292
zip_98030         0.1153      0.035      3.263      0.001       0.046       0.185
zip_98031         0.2119      0.034      6.220      0.000       0.145       0.279
zip_98032         0.0282      0.045      0.623      0.534      -0.060       0.117
zip_98033         1.5750      0.030     52.384      0.000       1.516       1.634
zip_98034         1.1244      0.029     38.586      0.000       1.067       1.181
zip_98038         0.3598      0.029     12.423      0.000       0.303       0.417
zip_98039         2.6177      0.063     41.660      0.000       2.495       2.741
zip_98040         1.8450      0.034     53.608   

In [33]:
coef_df=model.params.reset_index()
coef_df.columns=["Column","Value"]

def calculate_price (sqft_living, yr_built, zipcode, grade, waterfront, view , sqft_lot, coef_df=coef_df):
    if view == '-':
        view='NONE'

    if waterfront:
        waterfront='WATERFRONT'
    else:
        waterfront='NONE'

    b0,b1,b2,b3,b4,b5,b6,b7 = get_coeff( yr_built,zipcode,grade, waterfront, view, coef_df)
    y=round( np.exp(b0 + b1*np.log(sqft_living) + b2 + b3 + b4 + b5 + b6 + b7*np.log(sqft_lot)) )

    print(y)
    return y,b0,b1,b2,b3,b4,b5,b6,b7



In [35]:
abs_test_results=[]
test_results=[]

df=pd.concat([x_test, y_test], axis=1)
for i in range(4000):
    X=np.array(df[['sqft_living','yr_built','zipcode','grade','waterfront','view', 'sqft_lot','price']].sample())[0]
    sqft_living,year,zipcode,grade,water,view,sqft_lot,price=X

    b0,b1,b2,b3,b4,b5,b6,b7 = get_coeff( year=year, zipcode=zipcode, grade=grade, water=water, view=view, coef_df=coef_df)
    y=round( np.exp(b0 + b1*np.log(sqft_living) + b2 + b3 + b4 + b5 + b6 + b7*np.log(sqft_lot)) )
   
    miss_abs= np.abs(100-(y/price*100))
    miss= 100-(y/price*100)
    test_results.append(miss)
    abs_test_results.append(miss_abs)

avg_miss=sum(test_results)/len(test_results)
avg_miss_abs=sum(abs_test_results)/len(abs_test_results)
print (avg_miss, avg_miss_abs)


99.99975949697333 99.99975949697333
